In [9]:
from keras import Model
from keras.models import Sequential, Layer
from keras.layers import Dense, MaxPool2D, Dropout,MaxPool2D, Activation,BatchNormalization, InputLayer, Embedding, Flatten, Conv2D,concatenate, MaxPooling2D, Input, TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
import keras.utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.layers import Flatten, LSTM
import numpy as np
from __future__ import absolute_import

In [3]:
data= np.random.rand(100,10).reshape(-1,10)
labels = np.array([x for x in range(100)])
print(labels)
print(data.shape)
k.set_image_data_format('channels_last')
k.image_data_format()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
(100, 10)


'channels_last'

In [5]:
def loss(y_true, y_pred):
    return 0

In [6]:
x = Input(shape=(10,))
layer = Dense(1)
y = layer(x)
frozen_model = Model(x, y)
frozen_model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
print(layer.get_weights())
frozen_model.fit(data, labels, epochs=2)  

AttributeError: 'int' object has no attribute 'get_shape'

In [10]:
from keras import objectives

def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)


class CustomRegularization(Layer):
    def __init__(self, **kwargs):
        super(CustomRegularization, self).__init__(**kwargs)

    def call(self ,x ,mask=None):
        ld=x[0]
        rd=x[1]
        bce = objectives.binary_crossentropy(ld, rd)
        loss2 = K.sum(bce)
        self.add_loss(loss2,x)
        #you can output whatever you need, just update output_shape adequately
        #But this is probably useful
        return bce

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0],1)

input_size= 100
output_dim = 1
x1 = Input(shape=(input_size,))
ld = Dense(128, activation='relu')(x1)
out1 = Dense(output_dim, activation='sigmoid')(ld)

x2 = Input(shape=(input_size,))
rd = Dense(128, activation='relu')(x2)
out2 = Dense(output_dim, activation='sigmoid')(rd)

cr = CustomRegularization()([ld,rd])

m = Model( [x1,x2], [out1,out2,cr])
m.compile( loss=[K.binary_crossentropy,K.binary_crossentropy,zero_loss], optimizer="adam")

nb_examples = 32
print(m.predict( [np.random.randn(nb_examples,input_size),np.random.randn(nb_examples,input_size)] ))
print(m.fit(  [np.random.randn(nb_examples,input_size),np.random.randn(nb_examples,input_size)], [np.random.randn(nb_examples,output_dim),np.random.randn(nb_examples,output_dim), np.random.randn(nb_examples,1) ] )) 

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:636: UserWarning: Class `__main__.CustomRegularization` defines `get_output_shape_for` but does not override `compute_output_shape`. If this is a Keras 1 layer, please implement `compute_output_shape` to support Keras 2.
  output_shape = self.compute_output_shape(input_shape)


[array([[ 0.72582871],
       [ 0.28471425],
       [ 0.6002332 ],
       [ 0.46365148],
       [ 0.34994259],
       [ 0.89599079],
       [ 0.72997087],
       [ 0.74258912],
       [ 0.64278418],
       [ 0.32229829],
       [ 0.55125409],
       [ 0.47787449],
       [ 0.31173599],
       [ 0.73212522],
       [ 0.6930238 ],
       [ 0.5269261 ],
       [ 0.59011596],
       [ 0.54808384],
       [ 0.40443987],
       [ 0.47430289],
       [ 0.48621607],
       [ 0.68247658],
       [ 0.74261653],
       [ 0.79891586],
       [ 0.72460163],
       [ 0.63959157],
       [ 0.56982249],
       [ 0.621876  ],
       [ 0.53899729],
       [ 0.70246041],
       [ 0.71059704],
       [ 0.69662386]], dtype=float32), array([[ 0.27628058],
       [ 0.31734282],
       [ 0.24562642],
       [ 0.47597477],
       [ 0.53593421],
       [ 0.17901909],
       [ 0.63817805],
       [ 0.49656457],
       [ 0.42685932],
       [ 0.52211517],
       [ 0.58103591],
       [ 0.06994256],
       [ 0.477